In [ ]:
# ostatnie piwo: https://www.polskikraft.pl/piwo/9740
# ostatni browar: https://www.polskikraft.pl/browar/411
# ostatni user: https://www.polskikraft.pl/profil/28348

In [ ]:
import os
dumpdir = '/content/drive/MyDrive/datasets/PolskiKraft'

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

# scraping

In [ ]:
!mkdir browary
!mkdir piwa

## breweries

In [ ]:
for i in tqdm(range(1, 413)):
    ! wget --quiet https://www.polskikraft.pl/browar/{i} -O browary/{i}.html

In [ ]:
!zip -q -r browary.zip browary
!cp browary.zip {dumpdir}
!ls -lh {dumpdir}

total 2.6M
-rw------- 1 root root 2.6M Apr 10 02:18 browary.zip


## beers

In [ ]:
# for i in tqdm(range(1, 9740)):
#     ! wget --quiet https://www.polskikraft.pl/piwo/{i} -O piwa/{i}.html

# this would take ages (8-9h), lets go somewhat parallel

In [ ]:
import subprocess
MAX = 9740

i = 511
pbar = tqdm(total=MAX)
pbar.update(i)
while True:
    N = 50
    ls_output = [subprocess.Popen([
        "wget", "--quiet",
        f"https://www.polskikraft.pl/piwo/{j}",
        "-O", f"piwa/{j}.html"
    ]) for j in range(i, i+N)]
    pbar.set_postfix({'tasks': 'queued'})
    [x.communicate() for x in ls_output] # async wait
    pbar.set_postfix({'tasks': 'received'})
    i += N
    pbar.update(N)
    if i > MAX:
        break
pbar.close()
# now it will take about 1.5h, seems fine.
# This should finish by 6:30, 10th of april.
# Lets hope no one uses their app so early on Saturdays.

In [ ]:
!ls -lth piwa | wc -l

9760


In [ ]:
!zip -q -r piwa.zip piwa
!cp piwa.zip {dumpdir}
!ls -lh {dumpdir}

total 283M
-rw------- 1 root root  13K Apr 10 14:29 beer_network.csv
-rw------- 1 root root 2.7K Apr 10 14:29 beers_metadata.csv
-rw------- 1 root root 2.6M Apr 10 02:18 browary.zip
-rw------- 1 root root 280M Apr 10 15:07 piwa.zip
-rw------- 1 root root 676K Apr 10 14:29 ratings.csv


In [ ]:
# for i in tqdm(range(1, 9740+1)):
#     if not os.path.exists(f'piwa/{i}.html'):
#         !wget --quiet https://www.polskikraft.pl/piwo/{i} -O piwa/{i}.html

# for i in tqdm(range(9740+1 9900)):
#     if os.path.exists(f'piwa/{i}.html'):
#         !rm piwa/{i}.html

In theory the data in user htmls is redundant to what we already have
there should be no need to scrap the user profiles.

It would be a huge burden on the PolskiBrowar.pl servers to 
handle 20k+ of our requests downloading user profiles
in order not to disrupt their service (it would take at least 3h, 
in the worst case 23h) I do not even implement 
the user-data extracting code.
At least for now, beware.